In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
import json
import torchvision.ops
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import Image
import cv2
import albumentations as A
from torchvision.ops import box_convert

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm
from torchvision.utils import draw_bounding_boxes

from pycocotools.coco import COCO
from albumentations.pytorch import ToTensorV2
import sys

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
def get_transforms(train=False):
    if train:
        transform = A.Compose([
            A.Resize(600, 600),
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.1),
            A.ColorJitter(p=0.1),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    else:
        transform = A.Compose([
            A.Resize(600, 600),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    return transform

In [ ]:
class UI_Element_Detection(datasets.VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        super().__init__(root, transforms, transform, target_transform)
        self.split = split
        self.coco = COCO(os.path.join(root, split, "annotations_train.json"))
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]

    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))

    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))

        boxes = [t['bbox'] + [t['category_id']] for t in target]
        if self.transforms is not None:
            transformed = self.transforms(image=image, bboxes=boxes)

        image = transformed['image']
        boxes = transformed['bboxes']

        new_boxes = []
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.tensor(new_boxes, dtype=torch.float32)
        category_id_to_index = {cat_id: idx for idx, (cat_id, cat_info) in enumerate(categories.items())}

        targ = {}
        targ['boxes'] = boxes
        targ['labels'] = torch.tensor([category_id_to_index[t['category_id']] for t in target], dtype=torch.int64)
        targ['image_id'] = torch.tensor([t['image_id'] for t in target])
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        targ['iscrowd'] = torch.tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        return image.div(255), targ

    def __len__(self):
        return len(self.ids)

In [ ]:
dataset_path = "/content/drive/My Drive/MASC_UI_dataset/MASC_UI/MASC_UI"

In [ ]:
#load classes
coco = COCO(os.path.join(dataset_path, "train", "annotations_train.json"))
categories = coco.cats
n_classes = len(categories.keys())
categories

In [ ]:
classes = [i[1]['name'] for i in categories.items()]
classes

In [ ]:
train_dataset = UI_Element_Detection(root=dataset_path, transforms=get_transforms(True))

In [ ]:
val_dataset = UI_Element_Detection(root=dataset_path, split="val", transforms=get_transforms(False))


In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F


def get_file_name(coco, image_id):
    return coco.loadImgs(image_id)[0]['file_name']


sample_index = 333
sample = train_dataset[sample_index]
img_tensor = sample[0] * 255
img_int = img_tensor.to(torch.uint8)


image_id = sample[1]['image_id'][0].item()
file_name = get_file_name(train_dataset.coco, image_id)


new_width, new_height = img_tensor.shape[2] * 2, img_tensor.shape[1] * 2
img_resized = F.resize(img_int, [new_height, new_width])


boxes_resized = sample[1]['boxes'] * torch.tensor([2, 2, 2, 2], dtype=torch.float32)


img_with_boxes = draw_bounding_boxes(
    img_resized, boxes_resized, [classes[i] for i in sample[1]['labels']], width=4
)


plt.figure(figsize=(10, 10))
plt.imshow(img_with_boxes.permute(1, 2, 0))

for i, label in enumerate(sample[1]['labels']):
    box = boxes_resized[i]
    category_id = label.item()
    category_name = classes[category_id]
    plt.text(box[0], box[1], f"{category_name}", fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

plt.axis('off')

print(f"File Name: {file_name}")

for i, label in enumerate(sample[1]['labels']):
    box = boxes_resized[i]
    category_id = label.item()
    print(f"Bounding Box {i}: {box}, Category ID: {category_id}, Category Name: {classes[category_id]}")

plt.show()

In [ ]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

# model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn)

In [ ]:
images,targets = next(iter(train_loader))
images = list(image for image in images)
targets = [{k:v for k, v in t.items()} for t in targets]
output = model(images, targets)

In [ ]:
device = torch.device("cuda")

In [ ]:
model = model.to(device)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.007, momentum=0.9, nesterov=True, weight_decay=1e-4)

In [ ]:
import sys

In [ ]:

def train_one_epoch(model, optimizer, loader, device, epoch):
    model.to(device)
    model.train()

    all_losses = []
    all_losses_dict = []

    for images, targets in tqdm(loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()

        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    all_losses_dict = pd.DataFrame(all_losses_dict)
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))

    return all_losses, all_losses_dict


In [ ]:
import torch
import time
import numpy as np
from pycocotools.coco import COCO
import os


num_epochs = 50

total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    train_one_epoch(model, optimizer, train_loader, device, epoch)


total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total training time: {total_duration:.2f} seconds")

In [ ]:
epoch_losses_df = pd.DataFrame(epoch_losses_dict)

plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), epoch_losses, label='Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), epoch_losses_df['loss_classifier'], label='Loss Classifier')
plt.plot(range(num_epochs), epoch_losses_df['loss_box_reg'], label='Loss Box Reg')
plt.plot(range(num_epochs), epoch_losses_df['loss_rpn_box_reg'], label='Loss RPN Box Reg')
plt.plot(range(num_epochs), epoch_losses_df['loss_objectness'], label='Loss Objectness')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Individual Loss Components Over Epochs')
plt.legend()
plt.show()


In [ ]:
def check_class_distribution(loader):
    class_counts = [0] * 5

    for _, targets in loader:
        for target in targets:
            labels = target['labels'].numpy()
            for label in labels:
                class_counts[label] += 1

    return class_counts

train_class_distribution = check_class_distribution(train_loader)
val_class_distribution = check_class_distribution(val_loader)

print(f"Train class distribution: {train_class_distribution}")
print(f"Validation class distribution: {val_class_distribution}")

In [ ]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import os

def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2, y2g)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

def calculate_precision_recall(all_boxes, all_scores, all_labels, true_boxes, true_labels, iou_threshold=0.5):
    if len(all_scores) == 0:
        return np.array([]), np.array([]), np.array([])

    sorted_indices = np.argsort(-np.array(all_scores))
    all_boxes = np.array(all_boxes)[sorted_indices]
    all_labels = np.array(all_labels)[sorted_indices]
    all_scores = np.array(all_scores)[sorted_indices]

    num_true_boxes = len(true_boxes)
    num_pred_boxes = len(all_boxes)

    true_positive = np.zeros(num_pred_boxes)
    false_positive = np.zeros(num_pred_boxes)
    detected_boxes = []

    for idx, pred_box in enumerate(all_boxes):
        max_iou = 0
        max_idx = -1
        for jdx, true_box in enumerate(true_boxes):
            if all_labels[idx] == true_labels[jdx] and jdx not in detected_boxes:
                current_iou = iou(pred_box, true_box)
                if current_iou > max_iou:
                    max_iou = current_iou
                    max_idx = jdx

        if max_iou >= iou_threshold:
            true_positive[idx] = 1
            detected_boxes.append(max_idx)
        else:
            false_positive[idx] = 1

    cum_tp = np.cumsum(true_positive)
    cum_fp = np.cumsum(false_positive)
    precision = cum_tp / (cum_tp + cum_fp) if (cum_tp + cum_fp).sum() > 0 else np.array([])
    recall = cum_tp / num_true_boxes if num_true_boxes > 0 else np.array([])

    return precision, recall, all_scores

def calculate_ap(precision, recall):
    if len(precision) == 0 or len(recall) == 0:
        return 0.0

    recall = np.concatenate(([0.0], recall, [1.0]))
    precision = np.concatenate(([0.0], precision, [0.0]))

    for i in range(len(precision) - 2, -1, -1):
        precision[i] = max(precision[i], precision[i + 1])

    indices = np.where(recall[1:] != recall[:-1])[0]
    average_precision = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])

    return average_precision

def evaluate_model(loader, model, device, class_names, iou_thresh=0.5):
    model.eval()
    all_detections = []
    all_annotations = []

    with torch.no_grad():
        for images, targets in loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for i, output in enumerate(outputs):
                scores = output['scores'].cpu().numpy()
                labels = output['labels'].cpu().numpy()
                boxes = output['boxes'].cpu().numpy()

                target_boxes = targets[i]['boxes'].cpu().numpy()
                target_labels = targets[i]['labels'].cpu().numpy()

                valid = scores > 0.5
                scores = scores[valid]
                labels = labels[valid]
                boxes = boxes[valid]

                all_detections.extend([(box, score, label) for box, score, label in zip(boxes, scores, labels)])
                all_annotations.extend([(box, label) for box, label in zip(target_boxes, target_labels)])

    if not all_detections:
        return [], [], [], 0

    pred_boxes, pred_scores, pred_labels = zip(*all_detections)
    true_boxes, true_labels = zip(*all_annotations)

    aps, precisions, recalls = [], [], []

    for class_id in range(1, n_classes):
        class_pred_boxes = [pred_boxes[i] for i in range(len(pred_labels)) if pred_labels[i] == class_id]
        class_pred_scores = [pred_scores[i] for i in range(len(pred_labels)) if pred_labels[i] == class_id]
        class_true_boxes = [true_boxes[i] for i in range(len(true_labels)) if true_labels[i] == class_id]

        precision, recall, _ = calculate_precision_recall(class_pred_boxes, class_pred_scores, [class_id]*len(class_pred_boxes), class_true_boxes, [class_id]*len(class_true_boxes), iou_threshold=iou_thresh)
        ap = calculate_ap(precision, recall)

        if len(precision) == 0:
            print(f"No predictions for {class_names[class_id-1]}.")
        if len(recall) == 0:
            print(f"No true positives for {class_names[class_id-1]}.")

        aps.append(ap)
        precisions.append(np.mean(precision) if len(precision) > 0 else float('nan'))
        recalls.append(np.mean(recall) if len(recall) > 0 else float('nan'))

    mean_ap = np.mean(aps)
    return precisions, recalls, aps, mean_ap

n_classes = len(categories.keys())
classes = [i[1]['name'] for i in categories.items()]

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

model = model.to(device)

num_epochs = 70

epoch_losses = []
epoch_losses_dict = []

total_start_time = time.time()

train_mAPs, val_mAPs = [], []
train_precisions, val_precisions = [], []
train_recalls, val_recalls = [], []

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    losses, losses_dict = train_one_epoch(model, optimizer, train_loader, device, epoch)

    train_precisions_epoch, train_recalls_epoch, train_aps, train_mean_ap = evaluate_model(train_loader, model, device, classes)
    train_mAPs.append(train_mean_ap)
    train_precisions.append(np.nanmean(train_precisions_epoch))
    train_recalls.append(np.nanmean(train_recalls_epoch))
    print(f"Epoch {epoch} - Train mAP@0.50: {train_mean_ap}")
    for class_id, (precision, recall, ap) in enumerate(zip(train_precisions_epoch, train_recalls_epoch, train_aps), start=1):
        print(f"{classes[class_id-1]} - Precision: {precision}, Recall: {recall}, AP: {ap}")

    val_precisions_epoch, val_recalls_epoch, val_aps, val_mean_ap = evaluate_model(val_loader, model, device, classes)
    val_mAPs.append(val_mean_ap)
    val_precisions.append(np.nanmean(val_precisions_epoch))
    val_recalls.append(np.nanmean(val_recalls_epoch))
    print(f"Epoch {epoch} - Validation mAP@0.50: {val_mean_ap}")
    for class_id, (precision, recall, ap) in enumerate(zip(val_precisions_epoch, val_recalls_epoch, val_aps), start=1):
        print(f"{classes[class_id-1]} - Precision: {precision}, Recall: {recall}, AP: {ap}")

    epoch_losses.append(np.mean(losses))
    epoch_losses_dict.append(losses_dict.mean())

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch} completed in {epoch_duration:.2f} seconds")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total training time: {total_duration:.2f} seconds")

epochs = range(num_epochs)
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(epochs, train_mAPs, label='Train mAP')
plt.plot(epochs, val_mAPs, label='Validation mAP')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('mAP over Epochs')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, train_precisions, label='Train Precision')
plt.plot(epochs, val_precisions, label='Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Precision over Epochs')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs, train_recalls, label='Train Recall')
plt.plot(epochs, val_recalls, label='Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.title('Recall over Epochs')
plt.legend()

train_f1_scores = [2 * (p * r) / (p + r) for p, r in zip(train_precisions, train_recalls)]
val_f1_scores = [2 * (p * r) / (p + r) for p, r in zip(val_precisions, val_recalls)]
plt.subplot(2, 2, 4)
plt.plot(epochs, train_f1_scores, label='Train F1 Score')
plt.plot(epochs, val_f1_scores, label='Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
epoch_losses_df = pd.DataFrame(epoch_losses_dict)

plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), epoch_losses, label='Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), epoch_losses_df['loss_classifier'], label='Loss Classifier')
plt.plot(range(num_epochs), epoch_losses_df['loss_box_reg'], label='Loss Box Reg')
plt.plot(range(num_epochs), epoch_losses_df['loss_rpn_box_reg'], label='Loss RPN Box Reg')
plt.plot(range(num_epochs), epoch_losses_df['loss_objectness'], label='Loss Objectness')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Individual Loss Components Over Epochs')
plt.legend()
plt.show()


In [ ]:
model.eval()
torch.cuda.empty_cache()

In [ ]:
test_dataset = UI_Element_Detection(root=dataset_path, split="test", transforms=get_transforms(False))

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [ ]:
img, _ = test_dataset[45]
img_int = torch.tensor(img*255, dtype=torch.uint8)
with torch.no_grad():
    prediction = model([img.to(device)])
    pred = prediction[0]

In [ ]:
fig = plt.figure(figsize=(14, 10))
plt.imshow(draw_bounding_boxes(img_int,
    pred['boxes'][pred['scores'] > 0.5],
    [classes[i] for i in pred['labels'][pred['scores'] > 0.5].tolist()], width=4
).permute(1, 2, 0))

In [ ]:
pip install easyocr

In [ ]:
import pandas as pd
from flask import Flask, request, jsonify

import torch
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from PIL import Image
import numpy as np
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import easyocr
from IPython.display import display, HTML
import ipywidgets as widgets


expert_dataset_path = '/content/drive/My Drive/MASC_UI_dataset/MASC_UI/MASC_UI/expert_dataset.xlsx'
expert_df = pd.read_excel(expert_dataset_path)
login_texts = expert_df['Login'].dropna().tolist()
home_texts = expert_df['Home'].dropna().tolist()
welcome_texts = expert_df['Welcome'].dropna().tolist()
profile_texts = expert_df['Profile'].dropna().tolist()

reader = easyocr.Reader(['en'])

def load_image(uploaded):
    image = Image.open(io.BytesIO(uploaded['content'])).convert("RGB")
    return image

def preprocess_image_inference(image, transform):
    image = np.array(image)
    transformed = transform(image=image)
    image_tensor = transformed['image']
    return image_tensor.float() / 255.0

def recognize_text(image, predictions, classes):
    recognized_texts = []
    boxes = predictions['boxes'][predictions['scores'] > 0.5].cpu().numpy()
    labels = predictions['labels'][predictions['scores'] > 0.5].cpu().numpy()

    for box, label in zip(boxes, labels):
        class_name = classes[label]
        if class_name == 'TextView':
            x_min, y_min, x_max, y_max = map(int, box)
            cropped_image = image[y_min:y_max, x_min:x_max]
            results = reader.readtext(cropped_image)
            text_lines = " ".join([result[1] for result in results])
            recognized_texts.append((text_lines, box, class_name))
            print("Detected text:", text_lines)
    return recognized_texts

def visualize_results(image_tensor, predictions, classes):
    img_np = (image_tensor * 255).to(torch.uint8).cpu().permute(1, 2, 0).numpy()
    recognized_texts = recognize_text(img_np, predictions, classes)

    fig = plt.figure(figsize=(7, 5))
    plt.imshow(draw_bounding_boxes(
        (image_tensor * 255).to(torch.uint8).cpu(),
        predictions['boxes'][predictions['scores'] > 0.5].cpu(),
        [classes[i] for i in predictions['labels'][predictions['scores'] > 0.5].tolist()],
        width=4).permute(1, 2, 0))
    plt.axis('off')
    plt.show()

    return recognized_texts

def check_ux_and_suggestions(detected_texts, labels):
    detected_words = set(" ".join([text for text, _, _ in detected_texts]).split())

    expert_words = set()
    for column_texts in [login_texts, home_texts, welcome_texts, profile_texts]:
        for text in column_texts:
            expert_words.update(text.split())

    if detected_words & expert_words:
        return "Good UX", None

    detected_labels = set(labels)

    if 'Input' in detected_labels and not any(word in detected_words for word in login_texts):
        return "Fault in UX - Login", login_texts

    if detected_labels == {'ImageView', 'TextView', 'Button'}:
        return "Fault in UX - Home", home_texts

    if detected_labels <= {'ImageView', 'Button'} and len(detected_labels) == 2:
        return "Fault in UX - Welcome", welcome_texts

    if not any(word in detected_words for word in profile_texts):
        return "Fault in UX - Home", home_texts

    return "Good UX", None

def load_and_predict(change):
    uploaded = list(file_picker.value.values())[0]
    if not uploaded:
        return

    new_image = load_image(uploaded)
    transform = A.Compose([
        A.Resize(600, 600),
        ToTensorV2()
    ])
    image_tensor = preprocess_image_inference(new_image, transform)

    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model([image_tensor])
        pred = predictions[0]

    recognized_texts = visualize_results(image_tensor, pred, classes)
    detected_labels = [classes[label] for label in pred['labels'][pred['scores'] > 0.5].cpu().numpy()]
    ux_result, suggestions = check_ux_and_suggestions(recognized_texts, detected_labels)
    if ux_result != "Good UX" and suggestions:
        result_html.value = f"{ux_result}<br><br>Suggestions:<br>" + "<br>".join(suggestions)
    else:
        result_html.value = ux_result

file_picker = widgets.FileUpload(accept='image/*', multiple=False, description="Upload Image", button_style='info')
file_picker.observe(load_and_predict, names='value')

result_html = widgets.HTML(value="", layout=widgets.Layout(width='100%', height='200px', border='solid 1px black', padding='10px'))

image_output = widgets.Output(layout=widgets.Layout(width='50%'))
result_output = widgets.VBox([
    widgets.HTML("<h2>UI Element Detection</h2>"),
    file_picker,
    widgets.HTML("<h3>UX Evaluation Result:</h3>"),
    result_html
], layout=widgets.Layout(align_items='center', width='50%', margin='0 auto'))

def load_and_predict(change):
    uploaded = list(file_picker.value.values())[0]
    if not uploaded:
        return

    new_image = load_image(uploaded)
    transform = A.Compose([
        A.Resize(600, 600),
        ToTensorV2()
    ])
    image_tensor = preprocess_image_inference(new_image, transform)

    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model([image_tensor])
        pred = predictions[0]

    with image_output:
        image_output.clear_output()
        recognized_texts = visualize_results(image_tensor, pred, classes)

    detected_labels = [classes[label] for label in pred['labels'][pred['scores'] > 0.5].cpu().numpy()]
    ux_result, suggestions = check_ux_and_suggestions(recognized_texts, detected_labels)
    if ux_result != "Good UX" and suggestions:
        result_html.value = f"{ux_result}<br><br>Suggestions:<br>" + "<br>".join(suggestions)
    else:
        result_html.value = ux_result

file_picker.observe(load_and_predict, names='value')

app_layout = widgets.HBox([
    image_output,
    result_output
], layout=widgets.Layout(align_items='center', width='100%', margin='10 auto'))

display(app_layout)
